<a href="https://colab.research.google.com/github/sneha4948/Federated-Learning-for-Supply-Chain-Sensor-Data-Processing/blob/main/LSTM_Imputation_with_Simulated_data_flower_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing FLOWER, other requirements

In [ ]:
!pip install -q "flwr[simulation]" flwr-datasets

In [ ]:
# you might see a warning after running the command below, this can be ignored
# if you are running this outside Colab, you probably need to adjust the command below
# !pip install torch==1.13.1+cpu torchvision==0.14.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
!pip install matplotlib

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
# Flower Simulation code
import flwr as fl
from flwr.common import Context

In [ ]:
# Load the dataset
data = pd.read_excel('Data.xlsx')

# Display the first few rows to understand the structure
print(data.head())


   Temperature1  Humidity1  co2ppm1  Temperature2  Humidity2  co2ppm2  \
0         22.00      69.17   1171.0         23.50      63.80   2691.0   
1         21.99      69.23   1167.0         23.50      63.75   2688.0   
2         21.95      69.26   1164.0         23.50      63.81   2684.0   
3         21.93      69.43   1161.0         23.46      63.81   2681.0   
4         21.92      69.42   1157.0         23.44      63.90   2680.0   

   Temperature3  Humidity3  co2ppm3  
0         22.50      60.49      903  
1         22.47      60.28      912  
2         22.44      60.14      930  
3         22.45      60.44      996  
4         22.45      60.46     1139  


In [ ]:
# Select the first 1000 rows
data = data.iloc[:1000]

# Select the last 3 columns
sensor3_data = data.iloc[:, -3:]

# Rename columns for clarity if necessary
sensor3_data.columns = ['Temperature', 'Humidity', 'CO2']

# Display the selected data
print(sensor3_data.head())


   Temperature  Humidity   CO2
0        22.50     60.49   903
1        22.47     60.28   912
2        22.44     60.14   930
3        22.45     60.44   996
4        22.45     60.46  1139


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sensor3_data.to_csv('sensor3_1000data.csv', index=False)
print("Dataset created and saved as sensor3_1000data.csv")

Dataset created and saved as sensor3_1000data.csv


In [ ]:
# Load dataset
data = pd.read_csv('sensor3_1000data.csv')

In [ ]:
# Introduce missingness in Temperature column
def introduce_missingness(data, column, missing_rate=0.1):
    data_with_missing = data.copy()
    np.random.seed(42)
    num_missing = int(missing_rate * len(data))
    missing_indices = np.random.choice(data.index, num_missing, replace=False)
    data_with_missing.loc[missing_indices, column] = np.nan
    return data_with_missing

data_with_missing = introduce_missingness(data, 'Temperature')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Separate features and target (Temperature is the target, others are features)
X = data_with_missing[['Humidity', 'CO2']]
y = data_with_missing['Temperature']

# Fill missing values in y for training purposes
y_filled = y.fillna(y.mean())

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_filled, test_size=0.2, random_state=42)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# LSTM for imputation

In [ ]:
import tensorflow as tf

def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(2, 1)),  # Two input features reshaped
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1)  # Predict a single value (Temperature)
    ])
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['mae']
    )
    return model

In [ ]:
from flwr.client import NumPyClient

class LSTMClient(NumPyClient):
    def __init__(self, model, X_train, y_train, X_test, y_test):
        self.model = model
        self.X_train = np.expand_dims(X_train.values, axis=-1)  # Reshape for LSTM
        self.y_train = y_train.values
        self.X_test = np.expand_dims(X_test.values, axis=-1)
        self.y_test = y_test.values

    def get_parameters(self, config=None):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.get_parameters(), len(self.X_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        predictions = self.model.predict(self.X_test)
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(self.y_test, predictions)
        corr = np.corrcoef(self.y_test, predictions.flatten())[0, 1]
        print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, Correlation: {corr}")
        return mae, len(self.X_test), {"mae": mae, "rmse": rmse, "corr": corr}

In [ ]:
def client_fn(context: Context):
    model = create_model()
    return NumPyClient.to_client(LSTMClient(model, X_train, y_train, X_test, y_test))

In [ ]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedavg', num_rounds=5)


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 20:16:32,362	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3987906969.0, 'memory': 7975813940.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 396ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
(ClientAppActor pid=132006) MSE: 217.9364876874858, RMSE: 14.76267210526217, MAE: 14.667730150008204, Correlation: 0.5850240291954877


(ClientAppActor pid=132006) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7c1964842b00> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=132007) /usr/local/lib/python3.10/dist-packages/keras/src/layer

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 400ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step [repeated 4x across cluster]
(ClientAppActor pid=132006) MSE: 217.9364876874858, RMSE: 14.76267210526217, MAE: 14.667730150008204, Correlation: 0.5850240291954877 [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=132007) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 6x across cluster]
(ClientAppActor pid=132007)   warnings.warn( [repeated 6x across cluster]
(ClientAppActor pid=132007) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f242ca42560> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that

...

(ClientAppActor pid=132007) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=132007)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=132007) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=132007)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 401ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step [repeated 5x across cluster]
(ClientAppActor pid=132007) MSE: 98.36148214220006, RMSE: 9.91773573665885, MAE: 9.776564457798004, Correlation: 0.6190924622075316 [repeated 5x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=132007) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 6x across cluster]
(ClientAppActor pid=132007)   warnings.warn( [repeated 6x across cluster]
(ClientAppActor pid=132007) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 4x across cluster]
(ClientAppActor pid=132007)   warnings.warn( [repeated 4x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 422ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step [repeated 5x across cluster]
(ClientAppActor pid=132006) MSE: 63.88750950294294, RMSE: 7.992966251833104, MAE: 7.815783739471435, Correlation: 0.42892957843965845 [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 79.75s
INFO :      	History (loss, distributed):
INFO :      		round 1: 14.667730150008204
INFO :      		round 2: 12.325109793329238
INFO :      		round 3: 9.776564457798006
INFO :      		round 4: 7.815783739471435
INFO :      		round 5: 6.7130290893554685
INFO :      


In [ ]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedprox', num_rounds=5)


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 20:18:17,347	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7975868007.0, 'object_store_memory': 3987934003.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 408ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step
(ClientAppActor pid=133063) MSE: 183.29145050508393, RMSE: 13.538517293451449, MAE: 13.434707572841644, Correlation: 0.529415147261645


(ClientAppActor pid=133063) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7d6d31ffb520> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=133063) /usr/local/lib/python3.10/dist-packages/keras/src/layer

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 434ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step [repeated 4x across cluster]
(ClientAppActor pid=133063) MSE: 183.29145050508393, RMSE: 13.538517293451449, MAE: 13.434707572841644, Correlation: 0.529415147261645 [repeated 4x across cluster]


(ClientAppActor pid=133063) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=133063)   warnings.warn( [repeated 5x across cluster]
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=133061) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7b11e2c7c820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that

...

(ClientAppActor pid=133061) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=133061)   warnings.warn( [repeated 5x across cluster]


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=133063) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=133063)   warnings.warn( [repeated 3x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=133063) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=133063)   warnings.warn( [repeated 3x across cluster]


1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 415ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step [repeated 3x across cluster]
(ClientAppActor pid=133061) MSE: 44.35251844008124, RMSE: 6.659768647639439, MAE: 6.445922737503051, Correlation: -0.22040606020901632 [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 75.95s
INFO :      	History (loss, distributed):
INFO :      		round 1: 13.434707572841644
INFO :      		round 2: 10.359009874248505
INFO :      		round 3: 7.949902158164978
INFO :      		round 4: 6.445922737503051
INFO :      		round 5: 5.453863253974914
INFO :      


In [ ]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedbuff', num_rounds=5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 20:19:49,534	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 429ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step
(ClientAppActor pid=134068) MSE: 209.50415561354035, RMSE: 14.474258378705983, MAE: 14.376647240900995, Correlation: -0.6442448989273065


(ClientAppActor pid=134068) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7cbf03ccb520> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=134068) /usr/local/lib/python3.10/dist-packages/keras/src/layer

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 418ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step [repeated 4x across cluster]
(ClientAppActor pid=134068) MSE: 209.50415561354035, RMSE: 14.474258378705983, MAE: 14.376647240900995, Correlation: -0.6442448989273065 [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=134067) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=134067)   warnings.warn( [repeated 7x across cluster]
(ClientAppActor pid=134067) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7bd1cd01c820> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that

...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=134068) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=134068)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=134067) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=134067)   warnings.warn( [repeated 3x across cluster]


1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 728ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step [repeated 5x across cluster]
(ClientAppActor pid=134068) MSE: 79.61773125518384, RMSE: 8.922876848594507, MAE: 8.7643910531044, Correlation: -0.6237941770698313 [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=134067) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 7x across cluster]
(ClientAppActor pid=134067)   warnings.warn( [repeated 7x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=134067) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 3x across cluster]
(ClientAppActor pid=134067)   warnings.warn( [repeated 3x across cluster]


1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 721ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step [repeated 3x across cluster]
(ClientAppActor pid=134067) MSE: 55.974230962246814, RMSE: 7.481592809171508, MAE: 7.291996360206603, Correlation: 0.6281319790056029 [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 75.46s
INFO :      	History (loss, distributed):
INFO :      		round 1: 14.376647240900995
INFO :      		round 2: 11.26427145757675
INFO :      		round 3: 8.7643910531044
INFO :      		round 4: 7.291996360206603
INFO :      		round 5: 6.32008729019165
INFO :      


In [ ]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedasync', num_rounds=5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 20:21:21,291	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7977045198.0, 'object_store

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 391ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step
(ClientAppActor pid=135086) MSE: 191.32841033072043, RMSE: 13.832151326916591, MAE: 13.730186319732665, Correlation: -0.6543977583138629


(ClientAppActor pid=135086) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=135086)   warnings.warn( [repeated 5x across cluster]
(ClientAppActor pid=135086) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7a313eac6b00> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/pyt

1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 396ms/step [repeated 5x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step [repeated 4x across cluster]
(ClientAppActor pid=135086) MSE: 191.32841033072043, RMSE: 13.832151326916591, MAE: 13.730186319732665, Correlation: -0.6543977583138629 [repeated 4x across cluster]


(ClientAppActor pid=135087) WARNING:tensorflow:5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7fa5ec882200> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
(ClientAppActor pid=135086) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=135086)   warnings.

...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step


(ClientAppActor pid=135087) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 5x across cluster]
(ClientAppActor pid=135087)   warnings.warn( [repeated 5x across cluster]
(ClientAppActor pid=135087) /usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead. [repeated 2x across cluster]
(ClientAppActor pid=135087)   warnings.warn( [repeated 2x across cluster]
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step [repeated 2x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 412ms/step
(ClientAppActor pid=135087) MSE: 52.10421452477809, RMSE: 7.218324911278106, MAE: 7.021726823234558, Correlation: 0.6228866773002059
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step
(ClientAppActor pid=135087) MSE: 39.526993344273826, RMSE: 6.287049653396561, MAE: 6.06007029094696, Correlation: -0.6519474837908554
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 429ms/step


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 99.17s
INFO :      	History (loss, distributed):
INFO :      		round 1: 13.730186319732665
INFO :      		round 2: 10.87413100042343
INFO :      		round 3: 8.47368652381897
INFO :      		round 4: 7.021726823234558
INFO :      		round 5: 6.06007029094696
INFO :      
